In [1]:
import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd

# 1. 리뷰 스크래핑

In [2]:
# 잡플래닛 로그인 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [3]:
# 1.정부/공공기관/공기업(1001), 2.은행/금융업(900), 3.네트워크/통신/모바일(704), 4.세무/회계(1004)
codes = ['1001', '900', '704', '1004']
nums = ['1', '2', '3', '4']

In [4]:
# 리뷰를 txt파일로 저장
f = open('review_train.txt', 'w', encoding = 'utf-8')

for code, count in zip(codes, nums):
    reviews = {}
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math
    pages = math.ceil(num / 10) # 한 페이지당 리뷰 10개로 구성됨
    
    for i in range(1, pages + 1): # 전체 페이지의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_ = 'us_label') # 한 줄 리뷰 추출

        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().replace('\r', ' ') for label in label[:]]
        
        # 별점이 포함된 div 추출
        star = soup.find_all('div', class_="us_star_m") 
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))

        # txt파일에 리뷰 쓰기
        for k in range(len(labels)):
            start = labels[k].find('"')
            f.write(labels[k][start+1:-1] + '  ')
            # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
            reviews[labels[k][start+1:-1]] = stars[k].replace('.0', '')
 
    # 동적변수를 할당하여 산업군별 리뷰를 각각 리스트로 저장(총 4개의 딕셔너리 생성)
    globals()['reviews' + str(count)] = reviews
    f.write('\n')
f.close()

In [5]:
# 산업군별 리뷰 4개의 딕셔너리를 하나로 합치기
reviews_all = [reviews1, reviews2, reviews3, reviews4]

In [6]:
# 각 산업군별 리뷰 개수 확인
print(len(reviews1), len(reviews2), len(reviews3), len(reviews4))

774 1062 342 374


In [7]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f:
    print(len(f.readlines()))

4


###  review_train.txt는 하나의 산업군 리뷰 전체를 한 줄로 스크래핑하여 총 4문장으로 이루어져 있음
 - 각 리뷰는 '  '로 구분하여 저장하였음.
 - -> 이는 soynlp의 클래스 사용 시 적합하도록 저장한 것임.
 - (한 줄을 하나의 문서로 인식, 한 문서 내에서 두 칸 띄어쓰기로 문장 구분)
 
### 전체 산업군과 별도로 각각의 산업군별으로도 분석해보기 위해 텍스트 파일을 4개로 분할


In [8]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f :
    data = f.read().split('\n')
    for i in range(4):
        a = 1 + i
        with open(f'review_{a}.txt', 'w+', encoding = 'utf-8') as txt:
            txt.write(data[i]) 

# 2. soynlp를 이용한 명사 추출기 적용
#### 목적: 말뭉치를 기반으로 학습된 기존의 형태소 분석기는 pos 태깅 시 복합명사를 각각의 명사로 태깅하거나 
#### 새로 등장하는 단어를 명사로 인식하지 못하는 문제가 있음. 이에 따라 문서집합의 학습을 통해 명사점수를 부여하고 명사가 아닌지 판단
 - L - [R]의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단
 - 해당 패키지는 세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set에서 명사 가능 점수가 학습되어 있음.
 - 명사 가능 점수의 범위: [-1, 1]
 - 통계 기반의 unsupervised 학습 방법 적용
 
#### 두 가지의 명사 추출기를 적용하여 봄

In [9]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [10]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'review_train.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

2672

## 1) LRNounExtractor
#### soynlp의 기본적인 명사 추출기 적용

In [11]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


#### 전체 리뷰 문장에서 단어를 추출하여 명사를 판단하는 학습 실행

In [12]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (609, 326) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 172 nouns are extracted
Wall time: 398 ms


In [13]:
# 빈도수 기준 상위 100개 단어
sort_nouns1 = sorted(nouns.items(), key=lambda x:-x[1].frequency)[:100]
print(sort_nouns1)

[('회사', NounScore_v1(frequency=598, score=0.6502674638554216, known_r_ratio=0.7757009345794392)), ('업무', NounScore_v1(frequency=255, score=0.8760207908496732, known_r_ratio=0.9272727272727272)), ('사람', NounScore_v1(frequency=169, score=0.5331877950310557, known_r_ratio=0.6145038167938931)), ('기업', NounScore_v1(frequency=152, score=0.8826821944444446, known_r_ratio=0.6428571428571429)), ('다양', NounScore_v1(frequency=140, score=0.9994517080291968, known_r_ratio=0.9785714285714285)), ('사람들', NounScore_v1(frequency=129, score=0.9179094239130433, known_r_ratio=1.0)), ('추천', NounScore_v1(frequency=118, score=0.9883593124999999, known_r_ratio=0.3137254901960784)), ('생각', NounScore_v1(frequency=116, score=0.9319368923076924, known_r_ratio=0.5803571428571429)), ('복지', NounScore_v1(frequency=115, score=0.9135924666666667, known_r_ratio=1.0)), ('경험', NounScore_v1(frequency=114, score=0.6032131643835617, known_r_ratio=0.7156862745098039)), ('하는', NounScore_v1(frequency=91, score=0.6539166, known_r

In [26]:
# 빈도수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns1):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.frequency), end = '')


    회사 (598.00)    업무 (255.00)    사람 (169.00)    기업 (152.00)    다양 (140.00)
   사람들 (129.00)    추천 (118.00)    생각 (116.00)    복지 (115.00)    경험 (114.00)
    하는 (91.00)    연봉 (86.00)   직원들 (86.00)    직원 (83.00)   안정적 (81.00)
    직장 (79.00)    문화 (74.00)    성장 (73.00)    근무 (71.00)    체계 (68.00)
    급여 (64.00)    경력 (61.00)    가능 (56.00)   워라밸 (54.00)    환경 (51.00)
    자유 (51.00)   워라벨 (51.00)    발전 (50.00)    본인 (50.00)    최고 (49.00)
   계약직 (49.00)    신입 (48.00)   공무원 (46.00)    사업 (45.00)    좋아 (44.00)
   보수적 (43.00)    조직 (42.00)    장점 (42.00)    이상 (41.00)   커리어 (40.00)
  공공기관 (40.00)    대우 (38.00)    싶은 (38.00)    무난 (37.00)    하나 (35.00)
    개인 (35.00)   수평적 (35.00)    시간 (33.00)    야근 (33.00)   정규직 (33.00)
   사무실 (33.00)    대표 (32.00)    노력 (32.00)    규모 (32.00)    기회 (32.00)
    가족 (32.00)    분들 (32.00)    위치 (31.00)    필요 (31.00)    운영 (31.00)
    부서 (31.00)    나름 (31.00)  업무강도 (31.00)  근무환경 (31.00)    능력 (30.00)
    함께 (30.00)    입사 (30.00)    느낌 (30.00)   있는곳 (30.00)    단점 (29

In [27]:
# 빈도수 * 명사점수 기준 상위 100개 단어
sort_nouns2 = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]
print(sort_nouns2)

[('회사', NounScore_v1(frequency=598, score=0.6502674638554216, known_r_ratio=0.7757009345794392)), ('업무', NounScore_v1(frequency=255, score=0.8760207908496732, known_r_ratio=0.9272727272727272)), ('다양', NounScore_v1(frequency=140, score=0.9994517080291968, known_r_ratio=0.9785714285714285)), ('기업', NounScore_v1(frequency=152, score=0.8826821944444446, known_r_ratio=0.6428571428571429)), ('사람들', NounScore_v1(frequency=129, score=0.9179094239130433, known_r_ratio=1.0)), ('추천', NounScore_v1(frequency=118, score=0.9883593124999999, known_r_ratio=0.3137254901960784)), ('생각', NounScore_v1(frequency=116, score=0.9319368923076924, known_r_ratio=0.5803571428571429)), ('복지', NounScore_v1(frequency=115, score=0.9135924666666667, known_r_ratio=1.0)), ('사람', NounScore_v1(frequency=169, score=0.5331877950310557, known_r_ratio=0.6145038167938931)), ('안정적', NounScore_v1(frequency=81, score=0.9992101428571428, known_r_ratio=0.5384615384615384)), ('직원들', NounScore_v1(frequency=86, score=0.842951777777777

In [28]:
# 빈도수 * 명사점수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns2):
    if i % 5 ==0:
        print()
    print('%6s (%.2f)' % (word, score.score), end = '')
f.close()


    회사 (0.65)    업무 (0.88)    다양 (1.00)    기업 (0.88)   사람들 (0.92)
    추천 (0.99)    생각 (0.93)    복지 (0.91)    사람 (0.53)   안정적 (1.00)
   직원들 (0.84)    경험 (0.60)    직장 (0.82)    문화 (0.84)    체계 (0.89)
    하는 (0.65)    성장 (0.81)    급여 (0.88)    가능 (0.99)    연봉 (0.64)
    자유 (0.99)    근무 (0.70)    본인 (0.95)    경력 (0.77)    최고 (0.96)
    신입 (0.91)    좋아 (0.99)   보수적 (1.00)    직원 (0.51)   커리어 (1.00)
    조직 (0.91)    환경 (0.75)    싶은 (1.00)    무난 (1.00)   공무원 (0.80)
    개인 (1.00)   수평적 (1.00)   계약직 (0.71)  공공기관 (0.86)    이상 (0.84)
    나름 (1.00)   워라벨 (0.61)    부서 (0.99)    필요 (0.99)    분들 (0.96)
   있는곳 (1.00)    기회 (0.93)    입사 (0.99)    함께 (0.98)    노력 (0.91)
    적당 (1.00)   전반적 (1.00)    느낌 (0.96)    규모 (0.89)    운영 (0.92)
    가족 (0.88)    야근 (0.85)   워라밸 (0.51)   사무실 (0.82)    위치 (0.88)
    기관 (1.00)    업계 (1.00)    부족 (0.99)    발전 (0.53)    대우 (0.69)
    대표 (0.82)    시간 (0.79)    계속 (1.00)   공기업 (0.88)    때문 (1.00)
    전문 (1.00)    장점 (0.59)    최악 (1.00)    차이 (1.00)    성과 (1.00)
    만족 (0

## 2) NewsNounExtractor
#### 뉴스 데이터에서 더 좋은 성능을 내기 위해 여러가지 후처리 과정이 추가된 명사 추출기 적용

In [29]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/hsmy1/Anaconda3/envs/py37/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [30]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (26540, 15822, 13049)
predicting noun score was done                                        
before postprocessing 3520
_noun_scores_ 1082
checking hardrules ... done / 1082떡볶+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 660
extracted 0 compounds from eojeols

In [31]:
# 각 단어별 점수 및 빈도수 확인
words = ['공공기관', '워라밸', '분위기', '야근', '꼰대']

for word in words:
    if not word in nouns:
        continue
    score = nouns[word]
    print('word: %s, score: %.2f, frequency: %.2f' % (word, score.score, score.frequency))

word: 공공기관, score: 0.89, frequency: 56.00
word: 워라밸, score: 0.52, frequency: 56.00
word: 분위기, score: 0.52, frequency: 237.00
word: 야근, score: 0.85, frequency: 47.00
word: 꼰대, score: 0.67, frequency: 28.00


### 각 산업군별 차이를 확인하기 위해 각 산업군별 리뷰 학습 실행

In [32]:
corpus_fname1 = 'review_1.txt'
sentences1 = DoublespaceLineCorpus(corpus_fname1, iter_sent=True)
len(sentences1)

815

In [33]:
corpus_fname2 = 'review_2.txt'
sentences2 = DoublespaceLineCorpus(corpus_fname2, iter_sent=True)
len(sentences2)

1114

In [34]:
corpus_fname3 = 'review_3.txt'
sentences3 = DoublespaceLineCorpus(corpus_fname3, iter_sent=True)
len(sentences3)

358

In [35]:
corpus_fname4 = 'review_4.txt'
sentences4 = DoublespaceLineCorpus(corpus_fname4, iter_sent=True)
len(sentences4)

385

In [45]:
# 리뷰 개수가 적은 산업군을 감안하여 최소 빈도수를 15로 조정
def nounextractor(sent):
    noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
    )
    
    nouns = noun_extractor.train_extract(
        sent,
        min_noun_score=0.3,
        min_noun_frequency=15
    )

    # 빈도수 * 명사점수 기준 상위 100개 단어
    sort_nouns = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]

    for i, (word, score) in enumerate(sort_nouns):
        if i % 5 ==0:
            print()
        print('%6s (%.2f, %.2f)' % (word, score.frequency, score.score), end = '')

In [47]:
nounextractor(sentences1)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (251, 127) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 52 nouns are extracted

    업무 (97.00, 0.84)   분위기 (62.00, 1.00)    다양 (40.00, 1.00)   사람들 (37.00, 0.88)    복지 (32.00, 0.98)
    직장 (36.00, 0.86)   계약직 (37.00, 0.81)    생각 (34.00, 0.88)    근무 (34.00, 0.85)   공무원 (38.00, 0.75)
  공공기관 (34.00, 0.82)   안정적 (28.00, 1.00)    추천 (27.00, 0.99)    경험 (44.00, 0.60)    기업 (24.00, 1.00)
    기관 (23.00, 1.00)    회사 (66.00, 0.34)    급여 (22.00, 0.92)    가능 (20.00, 0.99)    조직 (21.00, 0.88)
    사람 (48.00, 0.38)   공기업 (18.00, 0.94)    환경 (21.00, 0.78)    서울 (15.00, 1.00)   워라밸 (24.00, 0.62)
    좋아 (15.00, 0.98)    체계 (19.00, 0.76)   정규직 (19.00, 0.72)   직원들 (16.00, 0.82)    만족 (13.00, 1.00)
    자유 (13.00, 0.95)    문화 (17.00, 0.71)    본인 (14.00, 0.87)    개인 (12.00, 1.00)   

In [48]:
nounextractor(sentences2)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (365, 187) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 82 nouns are extracted

    회사 (320.00, 0.64)    업무 (87.00, 0.93)    기업 (77.00, 0.85)   사람들 (61.00, 0.93)    다양 (48.00, 1.00)
    추천 (45.00, 0.98)    생각 (44.00, 0.99)   안정적 (40.00, 1.00)    연봉 (58.00, 0.68)    문화 (43.00, 0.89)
    복지 (45.00, 0.85)    사람 (59.00, 0.64)    성장 (43.00, 0.81)   직원들 (40.00, 0.79)    직장 (31.00, 0.92)
   보수적 (28.00, 1.00)    최고 (29.00, 0.93)    하는 (40.00, 0.65)    본인 (26.00, 0.98)    체계 (26.00, 0.97)
   커리어 (24.00, 1.00)    은행 (25.00, 0.93)    경험 (37.00, 0.63)    급여 (26.00, 0.83)    직원 (34.00, 0.64)
    가능 (20.00, 0.99)    규모 (22.00, 0.88)    무난 (19.00, 1.00)    싶은 (19.00, 1.00)    경력 (26.00, 0.73)
    투자 (22.00, 0.85)    이상 (21.00, 0.86)    업계 (18.00, 1.00)    기회 (18.00, 0.99)  

In [49]:
nounextractor(sentences3)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (99, 60) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 13 nouns are extracted

    회사 (121.00, 0.89)    기업 (38.00, 0.86)    업무 (32.00, 0.80)    다양 (21.00, 1.00)    직원 (24.00, 0.73)
    복지 (18.00, 0.95)    생각 (11.00, 0.94)    성장 (14.00, 0.69)    경험 (15.00, 0.61)    사람 (17.00, 0.52)
    연봉 (13.00, 0.47)    사업 (10.00, 0.55)    근무 (7.00, 0.63)

In [50]:
nounextractor(sentences4)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (114, 62) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 17 nouns are extracted

    회사 (77.00, 0.55)    업무 (36.00, 0.90)    다양 (29.00, 1.00)    사람 (35.00, 0.70)    신입 (21.00, 0.99)
   사무실 (19.00, 1.00)   사람들 (19.00, 0.96)   직원들 (17.00, 0.95)   회계사 (13.00, 0.97)   거래처 (15.00, 0.74)
    야근 (11.00, 1.00)    경력 (15.00, 0.67)    입사 (9.00, 0.99)    경험 (13.00, 0.49)   세무사 (11.00, 0.46)
    개인 (5.00, 1.00)    생각 (6.00, 0.80)